In [1]:
%cd /kaggle
!mkdir -p temp/dataset/train temp/dataset/val temp/dataset/test

%cd /kaggle/temp/dataset/train
!mkdir MEL NV BCC AKIEC BKL DF VASC 
%ls

%cd /kaggle/temp/dataset/val
!mkdir MEL NV BCC AKIEC BKL DF VASC 
%ls

%cd /kaggle/temp/dataset/test
!mkdir MEL NV BCC AKIEC BKL DF VASC 
%ls

%cd /kaggle/temp/

/kaggle
/kaggle/temp/dataset/train
AKIEC/  BCC/  BKL/  DF/  MEL/  NV/  VASC/
/kaggle/temp/dataset/val
AKIEC/  BCC/  BKL/  DF/  MEL/  NV/  VASC/
/kaggle/temp/dataset/test
AKIEC/  BCC/  BKL/  DF/  MEL/  NV/  VASC/
/kaggle/temp


In [2]:
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

In [3]:
imgs = glob('/kaggle/input/ham1000-segmentation-and-classification/images/*.jpg')

In [4]:
annot = pd.read_csv('/kaggle/input/ham1000-segmentation-and-classification/GroundTruth.csv')

In [5]:
X_train_val, X_test = train_test_split(annot,train_size =0.9, shuffle = True, random_state = 8)

In [6]:
X_train, X_val = train_test_split(X_train_val,train_size =0.8, shuffle = True, random_state = 8)

In [7]:
cls = X_train.columns[1:]
cls

Index(['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'], dtype='object')

In [8]:
for cl in cls:
    imgs_name = X_train[X_train[cl]==1]['image'].to_list()
    for img in imgs_name:
        shutil.copy(f'/kaggle/input/ham1000-segmentation-and-classification/images/{img}.jpg', 
                   f'/kaggle/temp/dataset/train/{cl}')
        
for cl in cls:
    imgs_name = X_val[X_val[cl]==1]['image'].to_list()
    for img in imgs_name:
        shutil.copy(f'/kaggle/input/ham1000-segmentation-and-classification/images/{img}.jpg', 
                   f'/kaggle/temp/dataset/val/{cl}')

for cl in cls:
    imgs_name = X_test[X_test[cl]==1]['image'].to_list()
    for img in imgs_name:
        shutil.copy(f'/kaggle/input/ham1000-segmentation-and-classification/images/{img}.jpg', 
                   f'/kaggle/temp/dataset/test/{cl}')

In [9]:
%ls /kaggle/temp/dataset/train/*/* | wc -l

%ls /kaggle/temp/dataset/val/*/* | wc -l

%ls /kaggle/temp/dataset/test/*/* | wc -l

7210
1803
1002


In [10]:
%cd /kaggle/temp/
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -qr requirements.txt

/kaggle/temp
Cloning into 'yolov5'...
remote: Enumerating objects: 16559, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 16559 (delta 18), reused 29 (delta 13), pack-reused 16522
Receiving objects: 100% (16559/16559), 15.08 MiB | 23.98 MiB/s, done.
Resolving deltas: 100% (11375/11375), done.
/kaggle/temp/yolov5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


In [11]:
!ls /kaggle/temp/

dataset  yolov5


In [12]:
!python classify/train.py --model yolov5s-cls.pt --data /kaggle/temp/dataset --epochs 3 --img 224 \
--project /kaggle/working/train --pretrained 'False'

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-04-18 09:12:05.044183: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 09:12:05.044322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 09:12:05.230494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
classi

In [13]:
!python classify/val.py --weights /kaggle/working/train/exp/weights/best.pt --data /kaggle/temp/dataset

classify/val: data=/kaggle/temp/dataset, weights=['/kaggle/working/train/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.13 torch-2.1.2+cpu CPU

Fusing layers... 
Model summary: 117 layers, 4175655 parameters, 0 gradients, 10.4 GFLOPs
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
testing: 100%|██████████| 8/8 [00:34<00:00,  4.35s/it]
                   Class      Images    top1_acc    top5_acc
                     all        1002       0.699       0.973
                   AKIEC          28           0           1
                     BCC          48           0           1
                     BKL         116       0.371           1
                     